In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

# **Parkinson's Disease Voice Dataset - Regression Task**

This dataset contains **5,875 voice recordings** from **42 early-stage Parkinson’s disease patients**.  
The goal is to **predict `motor_UPDRS`**, a clinical score for movement symptoms like **stiffness or tremors**, using **16 voice features** and other patient details.

---

## **Dataset Overview**
- **Total Patients:** 42  
- **Total Samples:** 5,875  
- **Prediction Target:** `motor_UPDRS` (0 to 100, higher = worse symptoms)  
- **Features Used for Prediction:**
  - **Demographics & Study Time:**
    - `age`: Patient’s age (years)
    - `sex`: Gender **(0 = male, 1 = female)**
    - `test_time`: Days since the patient joined the study (tracks symptom progression)

  - **Voice Features (16 Total)**
    - **Jitter Measures** *(Voice pitch stability, higher = more unstable)*
      - `Jitter(%)`, `Jitter(Abs)`, `Jitter:RAP`, `Jitter:PPQ5`, `Jitter:DDP`
    
    - **Shimmer Measures** *(Voice volume stability, higher = more unstable)*
      - `Shimmer`, `Shimmer(dB)`, `Shimmer:APQ3`, `Shimmer:APQ5`, `Shimmer:APQ11`, `Shimmer:DDA`
    
    - **Noise & Complexity Measures**
      - `NHR`: Noise-to-harmonics ratio **(higher = noisier, hoarse voice)**
      - `HNR`: Harmonics-to-noise ratio **(higher = clearer voice)**
      - `RPDE`: Recurrence Period Density Entropy **(higher = less predictable voice patterns)**
      - `DFA`: Detrended Fluctuation Analysis **(higher = more random voice patterns)**
      - `PPE`: Pitch Period Entropy **(higher = unstable pitch)**

---

## **Target Variable**
- `motor_UPDRS`: **Clinician-given score** from **0 to 100** (higher = worse movement symptoms).

### **Example Relationship**
- A patient with **high `Jitter(%)` and `Shimmer(dB)`** may have a **higher `motor_UPDRS` score**, indicating **worse movement symptoms**.

---

## **Target Variable**
- `motor_UPDRS`: **Clinician-given score** from **0 to 100** (higher = worse movement symptoms).

### **Example Relationship**
- A patient with **high `Jitter(%)` and `Shimmer(dB)`** may have a **higher `motor_UPDRS` score**, indicating **worse movement symptoms**.

---

## **How to Use This Data**
- **Train a regression model** to predict `motor_UPDRS` based on **16 voice features, age, sex, and test_time**.
-Each patient has **multiple recordings over time**, allowing us to **track symptom progression**.


<div style="text-align:center;font-size:30px; ont-weight:bold;">Loading and exploring the dataset</div>

In [3]:
df  = pd.read_csv('dataset_regression/parkinsons_updrs.data')
df.head()

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
0,1,72,0,5.6431,28.199,34.398,0.00662,0.000034,0.00401,0.00317,...,0.230,0.01438,0.01309,0.01662,0.04314,0.014290,21.640,0.41888,0.54842,0.16006
1,1,72,0,12.6660,28.447,34.894,0.00300,0.000017,0.00132,0.00150,...,0.179,0.00994,0.01072,0.01689,0.02982,0.011112,27.183,0.43493,0.56477,0.10810
2,1,72,0,19.6810,28.695,35.389,0.00481,0.000025,0.00205,0.00208,...,0.181,0.00734,0.00844,0.01458,0.02202,0.020220,23.047,0.46222,0.54405,0.21014
3,1,72,0,25.6470,28.905,35.810,0.00528,0.000027,0.00191,0.00264,...,0.327,0.01106,0.01265,0.01963,0.03317,0.027837,24.445,0.48730,0.57794,0.33277
4,1,72,0,33.6420,29.187,36.375,0.00335,0.000020,0.00093,0.00130,...,0.176,0.00679,0.00929,0.01819,0.02036,0.011625,26.126,0.47188,0.56122,0.19361


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5875 entries, 0 to 5874
Data columns (total 22 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   subject#       5875 non-null   int64  
 1   age            5875 non-null   int64  
 2   sex            5875 non-null   int64  
 3   test_time      5875 non-null   float64
 4   motor_UPDRS    5875 non-null   float64
 5   total_UPDRS    5875 non-null   float64
 6   Jitter(%)      5875 non-null   float64
 7   Jitter(Abs)    5875 non-null   float64
 8   Jitter:RAP     5875 non-null   float64
 9   Jitter:PPQ5    5875 non-null   float64
 10  Jitter:DDP     5875 non-null   float64
 11  Shimmer        5875 non-null   float64
 12  Shimmer(dB)    5875 non-null   float64
 13  Shimmer:APQ3   5875 non-null   float64
 14  Shimmer:APQ5   5875 non-null   float64
 15  Shimmer:APQ11  5875 non-null   float64
 16  Shimmer:DDA    5875 non-null   float64
 17  NHR            5875 non-null   float64
 18  HNR     

In [8]:
# Checking for duplicates
df.duplicated().sum()

0

In [6]:
df.describe()

,subject#,age,sex,test_time,motor_UPDRS,total_UPDRS,Jitter(%),Jitter(Abs),Jitter:RAP,Jitter:PPQ5,...,Shimmer(dB),Shimmer:APQ3,Shimmer:APQ5,Shimmer:APQ11,Shimmer:DDA,NHR,HNR,RPDE,DFA,PPE
count,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,...,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000,5875.000000
mean,21.494128,64.804936,0.317787,92.863722,21.296229,29.018942,0.006154,0.000044,0.002987,0.003277,...,0.310960,0.017156,0.020144,0.027481,0.051467,0.032120,21.679495,0.541473,0.653240,0.219589
std,12.372279,8.821524,0.465656,53.445602,8.129282,10.700283,0.005624,0.000036,0.003124,0.003732,...,0.230254,0.013237,0.016664,0.019986,0.039711,0.059692,4.291096,0.100986,0.070902,0.091498
min,1.000000,36.000000,0.000000,-4.262500,5.037700,7.000000,0.000830,0.000002,0.000330,0.000430,...,0.026000,0.001610,0.001940,0.002490,0.004840,0.000286,1.659000,0.151020,0.514040,0.021983
25%,10.000000,58.000000,0.000000,46.847500,15.000000,21.371000,0.003580,0.000022,0.001580,0.001820,...,0.175000,0.009280,0.010790,0.015665,0.027830,0.010955,19.406000,0.469785,0.596180,0.156340
50%,22.000000,65.000000,0.000000,91.523000,20.871000,27.576000,0.004900,0.000035,0.002250,0.002490,...,0.253000,0.013700,0.015940,0.022710,0.041110,0.018448,21.920000,0.542250,0.643600,0.205500
75%,33.000000,72.000000,1.000000,138.445000,27.596500,36.399000,0.006800,0.000053,0.003290,0.003460,...,0.365000,0.020575,0.023755,0.032715,0.061735,0.031463,24.444000,0.614045,0.711335,0.264490
max,42.000000,85.000000,1.000000,215.490000,39.511000,54.992000,0.099990,0.000446,0.057540,0.069560,...,2.107000,0.162670,0.167020,0.275460,0.488020,0.748260,37.875000,0.966080,0.865600,0.731730
